In [3]:
import pandas as pd
import numpy as np
import ast #used for literal_eval method in preprocessing
from sklearn.feature_extraction.text import CountVectorizer #For extracting tags and filtering out stop words
import nltk #For stemming and removing pseudo duplicate words

In [4]:
songs = pd.read_csv(r'C:\Users\Aravind Kumar\Documents\Programming Projects\KaggleDatasetMusicMLRec\dataset.csv')

In [5]:
songs = songs[['index','track_name','artists','album_name','duration','danceability','energy','key','loudness','speechiness','acousticness','liveness','valence','tempo','track_genre']]

In [6]:
songs.isnull().sum()
songs.dropna(inplace=True)

In [7]:
songs['duration'] = songs['duration'].apply(lambda x:round(x/1000))

In [8]:
songs.head()

,index,track_name,artists,album_name,duration,danceability,energy,key,loudness,speechiness,acousticness,liveness,valence,tempo,track_genre
0,0,Comedy,Gen Hoshino,Comedy,231,0.676,0.4610,1,-6.746,0.1430,0.0322,0.3580,0.715,87.917,acoustic
1,1,Ghost - Acoustic,Ben Woodward,Ghost (Acoustic),150,0.420,0.1660,1,-17.235,0.0763,0.9240,0.1010,0.267,77.489,acoustic
2,2,To Begin Again,Ingrid Michaelson;ZAYN,To Begin Again,211,0.438,0.3590,0,-9.734,0.0557,0.2100,0.1170,0.120,76.332,acoustic
3,3,Can't Help Falling In Love,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,202,0.266,0.0596,0,-18.515,0.0363,0.9050,0.1320,0.143,181.740,acoustic
4,4,Hold On,Chord Overstreet,Hold On,199,0.618,0.4430,2,-9.681,0.0526,0.4690,0.0829,0.167,119.949,acoustic
